# Storing Storm Data

The International Hurricane Watchgroup (IHW) has been asked to update their analysis tools. Because of the increase in public awareness of hurricanes, they are required to be more diligient with the analysis of historical hurricane data they share across the organization. 

The team of IHW have been having trouble sharing data across the teams and keeping it consistent. It seems that their method of sharing the data with their data anaylsts has been to save a CSV file on their local servers and have every data analyst pull the data down. Then, each analyst uses a local SQLite engine to store the CSV, run their queries, and send their results around.

Their CSV file contains the following fields:

* **fid** - ID for the row
* **year** - Recorded year
* **month** - Recorded month
* **day** - Recorded date
* **ad_time** - Recorded time in UTC
* **btid** - Hurricane ID
* **name** - Name of the hurricane
* **lat** - Latitude of the recorded location
* **long** - Longitude of the recorded location
* **wind_kts** - Wind speed in knots per second
* **pressure** - Atmospheric pressure of the hurricane
* **cat** - Hurricane category
* **basin** - The basin the hurricane is located
* **shape_leng** - Hurricane shape length

The goal is to create a database that will accomplish the following requirements:

* Database for the IHW to store their tables.
* Table that contains the fields detailed in the CSV file
* User that can update, read, and insert into a table of the data.
* Insert the data into the table.

## Download the CSV File

In [1]:
import csv
import io
from urllib import request
import pandas as pd

response = request.urlopen('https://dq-content.s3.amazonaws.com/251/storm_data.csv')
reader = csv.reader(io.TextIOWrapper(response))

count = 10
for line in reader:
    if count == 0:
        break
    else:
        print(line)
        count -= 1

['\ufeffFID', 'YEAR', 'MONTH', 'DAY', 'AD_TIME', 'BTID', 'NAME', 'LAT', 'LONG', 'WIND_KTS', 'PRESSURE', 'CAT', 'BASIN', 'Shape_Leng']
['2001', '1957', '8', '8', '1800Z', '63', 'NOTNAMED', '22.5', '-140', '50', '0', 'TS', 'Eastern Pacific', '1.140175']
['2002', '1961', '10', '3', '1200Z', '116', 'PAULINE', '22.1', '-140.2', '45', '0', 'TS', 'Eastern Pacific', '1.16619']
['2003', '1962', '8', '29', '0600Z', '124', 'C', '18', '-140', '45', '0', 'TS', 'Eastern Pacific', '2.10238']
['2004', '1967', '7', '14', '0600Z', '168', 'DENISE', '16.6', '-139.5', '45', '0', 'TS', 'Eastern Pacific', '2.12132']
['2005', '1972', '8', '16', '1200Z', '251', 'DIANA', '18.5', '-139.8', '70', '0', 'H1', 'Eastern Pacific', '1.702939']
['2006', '1976', '7', '22', '0000Z', '312', 'DIANA', '18.6', '-139.8', '30', '0', 'TD', 'Eastern Pacific', '1.6']
['2007', '1978', '8', '26', '1200Z', '342', 'KRISTY', '21.4', '-140.2', '45', '0', 'TS', 'Eastern Pacific', '1.30384']
['2008', '1980', '9', '24', '1800Z', '371', 'KA

## Explore the Dataset

In the `FID` column numbers vary between 2001 and on. From first observation, it already exceeds the small integer type. This column fits within the INTEGER size.

The last column, shape length, seems to keep the same precision of 1 digit before the decimal and 6 digits after.  Using DECIMAL can specify a max scale and precision that will ensure the data stays within the ranges.

## Create the Table

In [2]:
import psycopg2
conn = psycopg2.connect(dbname='lucy', user='lucy')
cur = conn.cursor()

cur.execute("""
   DROP TABLE IF EXISTS storm_data
""")

cur.execute("""
   CREATE TABLE storm_data (
       fid INTEGER PRIMARY KEY,
       datetime TIMESTAMP with time zone,
       btid INTEGER,
       name TEXT,
       lat DECIMAL(4,1), 
       long DECIMAL(4,1),
       wind_kts SMALLINT,
       pressure SMALLINT,
       cat VARCHAR(2),
       basin TEXT,
       shape_leng DECIMAL(8,6)
   )
""")
conn.commit()
cur.execute('SELECT * FROM storm_data LIMIT 0')
print(pd.Series(cur.description))
conn.close()

0            (fid, 23, None, 4, None, None, None)
1     (datetime, 1184, None, 8, None, None, None)
2           (btid, 23, None, 4, None, None, None)
3          (name, 25, None, -1, None, None, None)
4                (lat, 1700, None, 4, 4, 1, None)
5               (long, 1700, None, 4, 4, 1, None)
6       (wind_kts, 21, None, 2, None, None, None)
7       (pressure, 21, None, 2, None, None, None)
8          (cat, 1043, None, 2, None, None, None)
9         (basin, 25, None, -1, None, None, None)
10        (shape_leng, 1700, None, 8, 8, 6, None)
dtype: object


##  Create the Users

With a table set up, it's now time to create a user on the Postgres database that can insert, update, and read the data but not delete. This is to make sure that someone who might get a hold of this user does not issue a destructive command.

In [3]:
conn = psycopg2.connect(dbname='lucy', user='lucy')
cur = conn.cursor()
cur.execute("""
    DROP GROUP IF EXISTS analyst;
    CREATE GROUP analyst NOLOGIN;
    REVOKE ALL ON storm_data FROM analyst;
    GRANT SELECT, INSERT, UPDATE ON storm_data TO analyst;
    DROP USER IF EXISTS john;
    CREATE USER john WITH CREATEDB PASSWORD 'watson' IN GROUP analyst;
""")
conn.commit()
conn.close()

In [4]:
conn_john = psycopg2.connect(dbname='lucy', user='john', password='watson')
cur_john = conn_john.cursor()
cur_john.execute("""
    SELECT
     table_schema || '.' || table_name
 FROM
     information_schema.tables
 WHERE
     table_type = 'BASE TABLE'
 AND
     table_schema NOT IN ('pg_catalog', 'information_schema');
""")
print(cur_john.fetchall())
conn_john.close()

[('public.storm_data',)]


## Insert the Data

In [5]:
from datetime import datetime

conn = psycopg2.connect(dbname='lucy', user='lucy')
cur = conn.cursor()

## take data from file with columns:
## FID', 'YEAR', 'MONTH', 'DAY', 'AD_TIME', 'BTID', 'NAME', 'LAT', 
##'LONG', 'WIND_KTS', 'PRESSURE', 'CAT', 'BASIN', 'Shape_Leng'
## insert into table with columns:
## fid, datetime, btid, name, lat, long, wind_kts, pressure, cat, basin, shape_leng
response = request.urlopen('https://dq-content.s3.amazonaws.com/251/storm_data.csv')
reader = csv.reader(io.TextIOWrapper(response))
next(reader)

for line in reader:
    datetime_data = datetime(int(line[1]), int(line[2]), int(line[3]), int(line[4][:2]), int(line[4][2:4]))
    row = line[5:]
    row.insert(0, line[0])
    row.insert(1, datetime_data)    
    cur.execute("INSERT INTO storm_data VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", row)

conn.commit()
conn.close()

In [6]:
conn_john = psycopg2.connect(dbname='lucy', user='john', password='watson')
cur_john = conn_john.cursor()
cur_john.execute("""
    SELECT * FROM storm_data LIMIT 5
""")
print(pd.DataFrame(cur_john.fetchall()))
conn_john.close()

     0                          1    2         3     4       5   6   7   8   \
0  2001  1957-08-08 18:00:00-07:00   63  NOTNAMED  22.5  -140.0  50   0  TS   
1  2002  1961-10-03 12:00:00-08:00  116   PAULINE  22.1  -140.2  45   0  TS   
2  2003  1962-08-29 06:00:00-07:00  124         C  18.0  -140.0  45   0  TS   
3  2004  1967-07-14 06:00:00-07:00  168    DENISE  16.6  -139.5  45   0  TS   
4  2005  1972-08-16 12:00:00-07:00  251     DIANA  18.5  -139.8  70   0  H1   

                9         10  
0  Eastern Pacific  1.140175  
1  Eastern Pacific  1.166190  
2  Eastern Pacific  2.102380  
3  Eastern Pacific  2.121320  
4  Eastern Pacific  1.702939  
